In [2]:
#Import necessary libraries
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd
from datetime import date

In [3]:
#Create a function to scrape the data from the FDA Website
#We will get company name, address and last FDA inspection date based on the FEI number that it is part of the URL
executable_path = {'executable_path': ChromeDriverManager().install()}    
def fda_info(fei):
    browser = Browser('chrome', **executable_path, headless=False)
    url = 'https://datadashboard.fda.gov/ora/firmprofile.htm?FEIs='+str(fei)
    browser.visit(url)
    time.sleep(10) #time < 9 seconds does not work
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    browser.quit()
    header_results = soup.find_all('h3')#company and fei are stored in 'h3', fei should match the fei from url
    inspection_results = soup.find_all('span',class_="ng-binding ng-scope")#this is the class for inspection info
    address_results = soup.find_all('div',class_="col-xs-12 col-sm-12 col-md-4 col-lg-4 firm_address")#class for address
    
#store the specific info that you are looking for in variables, according to their indexes
#FEI number, Company name and last inspection date by FDA
    last_inspection = inspection_results[1].text
    fei = header_results[0].text
    company = header_results[1].text
    address = address_results[0].text

#return the info from the function
    return fei,company,last_inspection,address


[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 90.0.4430
[WDM] - Get LATEST driver version for 90.0.4430
[WDM] - Driver [C:\Users\meria\.wdm\drivers\chromedriver\win32\90.0.4430.24\chromedriver.exe] found in cache


In [4]:
#get the csv file information and convert it into a pandas dataframe
supplier_list = pd.read_csv('supplier_info.csv')
supplier_list.head()

,supplier_id,fei,brc
0,1001,2244191,1851425
1,1002,2412472,1657536
2,1003,1310769,6950748
3,1004,1046390,n
4,1005,1819038,5316427


In [5]:
#Collect the fei number from the csv file and create a loop to get the web information using the funtion fda_info;
supplier_fei = supplier_list['fei'].tolist()
fdaweb = []
for x in supplier_fei:
    try:    
        fdaweb.append(fda_info(x))
    except:
        pass
#print (fdaweb)

In [6]:
#Convert the FDA data into a Pandas DataFrame
df = pd.DataFrame(fdaweb)
FEI_df = df.rename(columns = {0:"fei",1:"Company Name_FDA",2:"Inspection Date_FDA",3:"Company Address_FDA"})
FEI_df

,fei,Company Name_FDA,Inspection Date_FDA,Company Address_FDA
0,2244191,AAK Foodservice,08/15/2019,"\nFirm Address\n465 Hillside Ave\nHillside, NJ..."
1,2412472,Acme Smoked Fish Corp.,03/12/2020,"\nFirm Address\n30 Gem St\nBrooklyn, NY 11222-..."
2,1310769,ADM Milling Company Inc.,01/29/2021,"\nFirm Address\n250 Ganson St\nBuffalo, NY 142..."
3,1046390,ADM Milling Company,06/16/2016,"\nFirm Address\n430 Central Ave Ne\nCleveland,..."
4,1819038,ADM Milling Company,11/02/2017,"\nFirm Address\n854 Bethel Ave\nBeech Grove, I..."
...,...,...,...,...
101,1410692,Newly Weds Foods Inc,08/02/2018,"\nFirm Address\n4140 W Fullerton Ave\nChicago,..."
102,1020094,Newly Weds Foods Inc,06/22/2018,"\nFirm Address\n5980 Hurt Rd\nHorn Lake, MS 38..."
103,3004252238,Cereal Ingredients Inc,05/11/2018,"\nFirm Address\n4900 S 13th St\nLeavenworth, K..."
104,3009327165,"Cuisine Innovations Unlimited, LLC",08/28/2019,"\nFirm Address\n180 Lehigh Ave\nLakewood, NJ 0..."


In [7]:
#Cleaning the address data in the address column
FEI_df['Company Address_FDA'] = FEI_df['Company Address_FDA'].str.replace('\nFirm Address\n','')
FEI_df['Company Address_FDA'] = FEI_df['Company Address_FDA'].str.replace('\n',' ')
FEI_df.head()

,fei,Company Name_FDA,Inspection Date_FDA,Company Address_FDA
0,2244191,AAK Foodservice,08/15/2019,"465 Hillside Ave Hillside, NJ 07205-1121 Unite..."
1,2412472,Acme Smoked Fish Corp.,03/12/2020,"30 Gem St Brooklyn, NY 11222-2804 United States"
2,1310769,ADM Milling Company Inc.,01/29/2021,"250 Ganson St Buffalo, NY 14203-3048 United St..."
3,1046390,ADM Milling Company,06/16/2016,"430 Central Ave Ne Cleveland, TN 37311-5590 Un..."
4,1819038,ADM Milling Company,11/02/2017,"854 Bethel Ave Beech Grove, IN 46107-1142 Unit..."


In [8]:
# Save the Pandas Data Frame in an excel file
file_name = 'FEI_webdata.xlsx'
FEI_df.to_excel(file_name)


In [10]:
#Create a function to scrape the data from the BRC Website
#We will get company name, address, certification grade and expiration date of the certificate

def brc_info(sitecode):
    browser = Browser('chrome', **executable_path, headless=False)
    url = 'https://directory.brcgs.com/site/'+str(sitecode)
    
    browser.visit(url)
    time.sleep(1) #time for the website do not think I am a robot
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    browser.quit()
        
    results = soup.find_all('h1')
    new_results = soup.find_all('li',class_="EntryPair_entryPair__1zyEr InfoBox_entryPair__35IoY")

    # Company name and address were in a fixed location 
    company = results[0].text
    address = new_results[1].text
    
    # Certification Grade, and Certificate exp date were stored in different index number, among the companies
    #In order to overcome this issue and keep using pandas, a text.partition was used to find the info imediattely after the key word (spl_word)
    for x in range(0,len(new_results)):
        spl_word="Grade"
        res = new_results[x].text.partition(spl_word)[2]
        if res != '':
            grade = res
            
    for x in range(0,len(new_results)):
        spl_word="Expiry Date"
        res = new_results[x].text.partition(spl_word)[2]
        if res != '':
            exp_date = res    
        
    return sitecode, company, address, grade, exp_date

In [11]:
#Collect the brc number from the csv file and creat a loop to get the web information using the funtion brc_info;
supplier_brccode = supplier_list['brc'].tolist()
brcweb = []
for x in supplier_brccode:
    if x != 'n':# when the supplier does not have a brc certification, it is listed as "n" and we don't need to use the function, code runs faster!
        try:
            brcweb.append(brc_info(x))
        except: #this except it is just in case htere is any issue with the brc code in the supplier list, i.e, if it is invalid or expired, the url will be different
            pass

In [12]:
#Conver the BRC data into a Pandas DataFrame
dfbrc = pd.DataFrame(brcweb)
BRC_df = dfbrc.rename(columns = {0:"brc",1:"BRC Company Name",2:"BRC Address", 3:"BRC Grade", 4: "BRC Expiration"})
BRC_df.head()

,brc,BRC Company Name,BRC Address,BRC Grade,BRC Expiration
0,1851425,AAK Foodservice,"Address465 Hillside Avenue, Hillside, New Jers...",C,2021-09-27
1,1657536,Acme Smoked Fish Corp,"Address30 Gem Street, Brooklyn, New York, New ...",AA,2022-04-06
2,6950748,ADM Milling Company - Buffalo,"Address250 Ganson St., Buffalo, New York, 1420...",A,2022-04-02
3,5316427,ADM Milling - Beech Grove,"Address854 Bethel Ave, Beech Grove, Indiana, 4...",AA,2021-11-16
4,1982979,ADM Milling - Charlotte,"Address620 W 10th St, Charlotte, North Carolin...",AA,2021-12-21


In [13]:
#Clean the address data in the address column
BRC_df['BRC Address'] = BRC_df['BRC Address'].str.replace('Address','')
BRC_df.head()


,brc,BRC Company Name,BRC Address,BRC Grade,BRC Expiration
0,1851425,AAK Foodservice,"465 Hillside Avenue, Hillside, New Jersey, 072...",C,2021-09-27
1,1657536,Acme Smoked Fish Corp,"30 Gem Street, Brooklyn, New York, New York, 1...",AA,2022-04-06
2,6950748,ADM Milling Company - Buffalo,"250 Ganson St., Buffalo, New York, 14203, UNIT...",A,2022-04-02
3,5316427,ADM Milling - Beech Grove,"854 Bethel Ave, Beech Grove, Indiana, 46107, U...",AA,2021-11-16
4,1982979,ADM Milling - Charlotte,"620 W 10th St, Charlotte, North Carolina, 2820...",AA,2021-12-21


In [14]:
# Save the Pandas Data Frame in an excel file
file = 'BRC_webdata.xlsx'
BRC_df.to_excel(file)

In [15]:
#convert columns to string before merging, so they have the same data type
BRC_df['brc']=BRC_df['brc'].apply(str)
FEI_df['fei']=FEI_df['fei'].apply(str)
supplier_list['fei']=supplier_list['fei'].apply(str)
supplier_list['brc']=supplier_list['brc'].apply(str)

In [16]:
#Merge the data
FEI_supplier = FEI_df.merge(supplier_list, how='outer', on ='fei')
FEI_BRC_supplier = FEI_supplier.merge(BRC_df, how='outer', on ='brc')


In [17]:
FEI_BRC_supplier.head()

,fei,Company Name_FDA,Inspection Date_FDA,Company Address_FDA,supplier_id,brc,BRC Company Name,BRC Address,BRC Grade,BRC Expiration
0,2244191,AAK Foodservice,08/15/2019,"465 Hillside Ave Hillside, NJ 07205-1121 Unite...",1001.0,1851425,AAK Foodservice,"465 Hillside Avenue, Hillside, New Jersey, 072...",C,2021-09-27
1,2412472,Acme Smoked Fish Corp.,03/12/2020,"30 Gem St Brooklyn, NY 11222-2804 United States",1002.0,1657536,Acme Smoked Fish Corp,"30 Gem Street, Brooklyn, New York, New York, 1...",AA,2022-04-06
2,1310769,ADM Milling Company Inc.,01/29/2021,"250 Ganson St Buffalo, NY 14203-3048 United St...",1003.0,6950748,ADM Milling Company - Buffalo,"250 Ganson St., Buffalo, New York, 14203, UNIT...",A,2022-04-02
3,1046390,ADM Milling Company,06/16/2016,"430 Central Ave Ne Cleveland, TN 37311-5590 Un...",1004.0,n,NaN,NaN,NaN,NaN
4,1416339,ADM Milling Co.,06/11/2019,"1300 W Carroll Ave Chicago, IL 60607-1118 Unit...",1006.0,n,NaN,NaN,NaN,NaN


In [18]:
#Convert FDA inspection date to date format
for i in range (len(FEI_BRC_supplier)) :
    
    try:
        FEI_BRC_supplier.loc[i, "Inspection Date_FDA"]= pd.to_datetime(FEI_BRC_supplier.loc[i,"Inspection Date_FDA"])
    except:
         FEI_BRC_supplier.loc[i, "Inspection Date_FDA"] = "Not available"
FEI_BRC_supplier.head()

,fei,Company Name_FDA,Inspection Date_FDA,Company Address_FDA,supplier_id,brc,BRC Company Name,BRC Address,BRC Grade,BRC Expiration
0,2244191,AAK Foodservice,2019-08-15 00:00:00,"465 Hillside Ave Hillside, NJ 07205-1121 Unite...",1001.0,1851425,AAK Foodservice,"465 Hillside Avenue, Hillside, New Jersey, 072...",C,2021-09-27
1,2412472,Acme Smoked Fish Corp.,2020-03-12 00:00:00,"30 Gem St Brooklyn, NY 11222-2804 United States",1002.0,1657536,Acme Smoked Fish Corp,"30 Gem Street, Brooklyn, New York, New York, 1...",AA,2022-04-06
2,1310769,ADM Milling Company Inc.,2021-01-29 00:00:00,"250 Ganson St Buffalo, NY 14203-3048 United St...",1003.0,6950748,ADM Milling Company - Buffalo,"250 Ganson St., Buffalo, New York, 14203, UNIT...",A,2022-04-02
3,1046390,ADM Milling Company,2016-06-16 00:00:00,"430 Central Ave Ne Cleveland, TN 37311-5590 Un...",1004.0,n,NaN,NaN,NaN,NaN
4,1416339,ADM Milling Co.,2019-06-11 00:00:00,"1300 W Carroll Ave Chicago, IL 60607-1118 Unit...",1006.0,n,NaN,NaN,NaN,NaN


In [19]:
# Insert new columns for the scores
FEI_BRC_supplier["brc_score"]=""
FEI_BRC_supplier["fda_score"]=""
FEI_BRC_supplier["Final Score"]=""
#insert column to hold today's date
FEI_BRC_supplier["Today"]=""

In [20]:
#insert the current date on Today column
for i in range(len(FEI_BRC_supplier)):
    FEI_BRC_supplier.loc[i,"Today"]=pd.to_datetime('today')

In [21]:
FEI_BRC_supplier.head()

,fei,Company Name_FDA,Inspection Date_FDA,Company Address_FDA,supplier_id,brc,BRC Company Name,BRC Address,BRC Grade,BRC Expiration,brc_score,fda_score,Final Score,Today
0,2244191,AAK Foodservice,2019-08-15 00:00:00,"465 Hillside Ave Hillside, NJ 07205-1121 Unite...",1001.0,1851425,AAK Foodservice,"465 Hillside Avenue, Hillside, New Jersey, 072...",C,2021-09-27,,,,2021-04-25 22:24:19.101569
1,2412472,Acme Smoked Fish Corp.,2020-03-12 00:00:00,"30 Gem St Brooklyn, NY 11222-2804 United States",1002.0,1657536,Acme Smoked Fish Corp,"30 Gem Street, Brooklyn, New York, New York, 1...",AA,2022-04-06,,,,2021-04-25 22:24:19.102601
2,1310769,ADM Milling Company Inc.,2021-01-29 00:00:00,"250 Ganson St Buffalo, NY 14203-3048 United St...",1003.0,6950748,ADM Milling Company - Buffalo,"250 Ganson St., Buffalo, New York, 14203, UNIT...",A,2022-04-02,,,,2021-04-25 22:24:19.102601
3,1046390,ADM Milling Company,2016-06-16 00:00:00,"430 Central Ave Ne Cleveland, TN 37311-5590 Un...",1004.0,n,NaN,NaN,NaN,NaN,,,,2021-04-25 22:24:19.102601
4,1416339,ADM Milling Co.,2019-06-11 00:00:00,"1300 W Carroll Ave Chicago, IL 60607-1118 Unit...",1006.0,n,NaN,NaN,NaN,NaN,,,,2021-04-25 22:24:19.102601


In [22]:
#calculate BRC score:
# BRC Score - Based on BRC Grade:
# If AA+ or AA = 0; A = 5; B=10; C=20; D=30
# If no BRC certification or certificate expired = 100 points

for i in range(len(FEI_BRC_supplier)):
    if FEI_BRC_supplier.loc[i,"BRC Grade"] == 'AA+':
        FEI_BRC_supplier.loc[i,"brc_score"]=0
    elif FEI_BRC_supplier.loc[i,"BRC Grade"] == 'AA':
        FEI_BRC_supplier.loc[i,"brc_score"]=0
    elif FEI_BRC_supplier.loc[i,"BRC Grade"] == 'A':
        FEI_BRC_supplier.loc[i,"brc_score"]=5
    elif FEI_BRC_supplier.loc[i,"BRC Grade"] == 'B':
        FEI_BRC_supplier.loc[i,"brc_score"]=10
    elif FEI_BRC_supplier.loc[i,"BRC Grade"] == str('C'):
        FEI_BRC_supplier.loc[i,"brc_score"]=20
    elif FEI_BRC_supplier.loc[i,"BRC Grade"] =='D':
        FEI_BRC_supplier.loc[i,"brc_score"]=30
    else:
        FEI_BRC_supplier.loc[i,"brc_score"]=100
        

In [23]:
FEI_BRC_supplier.head()

,fei,Company Name_FDA,Inspection Date_FDA,Company Address_FDA,supplier_id,brc,BRC Company Name,BRC Address,BRC Grade,BRC Expiration,brc_score,fda_score,Final Score,Today
0,2244191,AAK Foodservice,2019-08-15 00:00:00,"465 Hillside Ave Hillside, NJ 07205-1121 Unite...",1001.0,1851425,AAK Foodservice,"465 Hillside Avenue, Hillside, New Jersey, 072...",C,2021-09-27,20,,,2021-04-25 22:24:19.101569
1,2412472,Acme Smoked Fish Corp.,2020-03-12 00:00:00,"30 Gem St Brooklyn, NY 11222-2804 United States",1002.0,1657536,Acme Smoked Fish Corp,"30 Gem Street, Brooklyn, New York, New York, 1...",AA,2022-04-06,0,,,2021-04-25 22:24:19.102601
2,1310769,ADM Milling Company Inc.,2021-01-29 00:00:00,"250 Ganson St Buffalo, NY 14203-3048 United St...",1003.0,6950748,ADM Milling Company - Buffalo,"250 Ganson St., Buffalo, New York, 14203, UNIT...",A,2022-04-02,5,,,2021-04-25 22:24:19.102601
3,1046390,ADM Milling Company,2016-06-16 00:00:00,"430 Central Ave Ne Cleveland, TN 37311-5590 Un...",1004.0,n,NaN,NaN,NaN,NaN,100,,,2021-04-25 22:24:19.102601
4,1416339,ADM Milling Co.,2019-06-11 00:00:00,"1300 W Carroll Ave Chicago, IL 60607-1118 Unit...",1006.0,n,NaN,NaN,NaN,NaN,100,,,2021-04-25 22:24:19.102601


In [24]:
# Claculate FDA Score - Based on the last FDA inspection:
# - If FDA inspection it is over 3 years (1095 days) - 50 points
# - If FDA inspection it is over 2 years (730 days) - 20 points
# - If FDA inspection it is less than 2 years (730 days) - 5 points
# - If no FDA inspection - 100 points


for i in range(len(FEI_BRC_supplier)):
    
    try:
        if int((FEI_BRC_supplier.loc[i,"Today"] - FEI_BRC_supplier.loc[i,"Inspection Date_FDA"]).days)>1095:
            FEI_BRC_supplier.loc[i,"fda_score"]=50
        elif int((FEI_BRC_supplier.loc[i,"Today"] - FEI_BRC_supplier.loc[i,"Inspection Date_FDA"]).days)>730:
            FEI_BRC_supplier.loc[i,"fda_score"]=20
        elif int((FEI_BRC_supplier.loc[i,"Today"] - FEI_BRC_supplier.loc[i,"Inspection Date_FDA"]).days)<=730:
            FEI_BRC_supplier.loc[i,"fda_score"]=5
    except:
        FEI_BRC_supplier.loc[i,"fda_score"]=100

In [25]:
FEI_BRC_supplier

,fei,Company Name_FDA,Inspection Date_FDA,Company Address_FDA,supplier_id,brc,BRC Company Name,BRC Address,BRC Grade,BRC Expiration,brc_score,fda_score,Final Score,Today
0,2244191,AAK Foodservice,2019-08-15 00:00:00,"465 Hillside Ave Hillside, NJ 07205-1121 Unite...",1001.0,1851425,AAK Foodservice,"465 Hillside Avenue, Hillside, New Jersey, 072...",C,2021-09-27,20,5,,2021-04-25 22:24:19.101569
1,2412472,Acme Smoked Fish Corp.,2020-03-12 00:00:00,"30 Gem St Brooklyn, NY 11222-2804 United States",1002.0,1657536,Acme Smoked Fish Corp,"30 Gem Street, Brooklyn, New York, New York, 1...",AA,2022-04-06,0,5,,2021-04-25 22:24:19.102601
2,1310769,ADM Milling Company Inc.,2021-01-29 00:00:00,"250 Ganson St Buffalo, NY 14203-3048 United St...",1003.0,6950748,ADM Milling Company - Buffalo,"250 Ganson St., Buffalo, New York, 14203, UNIT...",A,2022-04-02,5,5,,2021-04-25 22:24:19.102601
3,1046390,ADM Milling Company,2016-06-16 00:00:00,"430 Central Ave Ne Cleveland, TN 37311-5590 Un...",1004.0,n,NaN,NaN,NaN,NaN,100,50,,2021-04-25 22:24:19.102601
4,1416339,ADM Milling Co.,2019-06-11 00:00:00,"1300 W Carroll Ave Chicago, IL 60607-1118 Unit...",1006.0,n,NaN,NaN,NaN,NaN,100,5,,2021-04-25 22:24:19.102601
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,1020094,Newly Weds Foods Inc,2018-06-22 00:00:00,"5980 Hurt Rd Horn Lake, MS 38637-3205 United S...",1103.0,2046429,"Newly Weds Foods, Inc. - Horn Lake","5980 Hurt Road, Horn Lake, Mississippi, 38637,...",A,2021-11-23,5,20,,2021-04-25 22:24:19.129666
104,3004252238,Cereal Ingredients Inc,2018-05-11 00:00:00,"4900 S 13th St Leavenworth, KS 66048 United St...",1104.0,6665737,"Cereal Ingredients, Inc.","4720 South 13th Street, Leavenworth, Kansas, 6...",A,2022-03-25,5,20,,2021-04-25 22:24:19.129666
105,3009327165,"Cuisine Innovations Unlimited, LLC",2019-08-28 00:00:00,"180 Lehigh Ave Lakewood, NJ 08701-4526 United ...",1105.0,1020958,"Cuisine Innovations Unlimited, LLC.","180 Lehigh Ave. Lakewood, New Jersey, New Jers...",A,2021-09-14,5,5,,2021-04-25 22:24:19.130816
106,3004804821,Innophos,2016-12-15 00:00:00,"10810 Highway 75 Geismar, LA 70734-3405 United...",1106.0,3604346,Innophos Inc.,"10810 Highway 75, Geismar, Louisiana, 70734, U...",AA,2022-04-26,0,50,,2021-04-25 22:24:19.130816


In [34]:
# Add bonth scores to get the supplier final score:
for i in range(len(FEI_BRC_supplier)):
    FEI_BRC_supplier.loc[i,"Final Score"] = FEI_BRC_supplier.loc[i,"fda_score"]+FEI_BRC_supplier.loc[i,"brc_score"]
#Sort by score
Final_table = FEI_BRC_supplier.sort_values(by='Final Score', ascending = True )
Final_table.head()

,fei,Company Name_FDA,Inspection Date_FDA,Company Address_FDA,supplier_id,brc,BRC Company Name,BRC Address,BRC Grade,BRC Expiration,brc_score,fda_score,Final Score,Today
93,1024675,The Pictsweet Company,2019-12-05 00:00:00,"10 Pictsweet Dr Bells, TN 38006-4274 United St...",1089.0,2044603,The Pictsweet Company - Bells,"10 Pictsweet Drive, Bells, Tennessee, 38006, U...",AA,2022-03-30,0,5,5,2021-04-25 22:24:19.126683
1,2412472,Acme Smoked Fish Corp.,2020-03-12 00:00:00,"30 Gem St Brooklyn, NY 11222-2804 United States",1002.0,1657536,Acme Smoked Fish Corp,"30 Gem Street, Brooklyn, New York, New York, 1...",AA,2022-04-06,0,5,5,2021-04-25 22:24:19.102601
66,3006675174,"Barry Callebaut USA, LLC",2020-01-13 00:00:00,"903 Industrial Hwy Eddystone, PA 19022-1531 Un...",1031.0,9433811,Barry Callebaut USA LLC (Eddystone Plant),"903 Industrial Highway, Eddystone, Pennsylvani...",AA,2021-11-24,0,5,5,2021-04-25 22:24:19.118592
99,3011193162,Mariani Nut Company,2019-11-13 00:00:00,"8 E Edwards St Winters, CA 95694 United States",1099.0,1195385,Mariani Nut Company,"12 E. Baker St, Winters, California, 95694, UN...",AA,2021-08-10,0,5,5,2021-04-25 22:24:19.128674
85,3003763602,Grain Processing Corporation,2019-08-08 00:00:00,"1443 S 300 W Washington, IN 47501-7410 United ...",1072.0,2005058,Grain Processing Corporation - Washington Malt...,"1443 S 300 W, Department 31, Washington, India...",AA,2021-08-29,0,5,5,2021-04-25 22:24:19.124634


In [35]:
Finalfile = 'Final_Suppier_Score.xlsx'
Final_table.to_excel(Finalfile)